# Provided code

You shouldn't need to change anything in this section.

### Load data to Colab

In [1]:
if False:  # manual loading
    from google.colab import file
    uploaded = files.upload()  # then browse, select the files
    
else:  # automatic loading
    import requests
    import gzip
    
    filepath_d_gr = 'http://users.diag.uniroma1.it/challenge9/data/USA-road-d/' + 'USA-road-d.NY.gr.gz'
    filepath_t_gr = 'http://users.diag.uniroma1.it/challenge9/data/USA-road-t/' + 'USA-road-t.NY.gr.gz'
    filepath_d_co = 'http://users.diag.uniroma1.it/challenge9/data/USA-road-d/' + 'USA-road-d.NY.co.gz'
    
    def loader(url):
        name = url.rsplit('/', 1)[1].rsplit('.', 1)[0]
        savename = name + '.txt'
        
        with open(savename, 'wb') as f_out:
            with requests.get(url) as r:
                f_in = gzip.decompress(r.content)
                f_out.write(f_in)
                
        print(savename)
            
    loader(filepath_d_gr)
    loader(filepath_t_gr)
    loader(filepath_d_co)

USA-road-d.NY.gr.txt
USA-road-t.NY.gr.txt
USA-road-d.NY.co.txt


### Graph and Vertex classes

In [2]:
# Vertex implementation
class Vertex:
    # Initialization of a vertex, given a neighbor and the corresponding weight
    # Each vertex contains a list of neighbors and corresponding weights
    def __init__(self, i, neighbor_index, weight):
        self.index = i
        self.neighbors = [neighbor_index]
        self.weights = [weight]
        
    def getNeighbors(self):
        return self.neighbors
    
    def getWeights(self):
        return self.weights
    
    # Add a neighbor with corresponding weight to the vertex
    def _addNeighbor(self, neighbor_index, weight):
        self.neighbors.append(neighbor_index)
        self.weights.append(weight)


# Graph data structure
class Graph:
    # Initializes a graph with n_vertices nodes
    # The graph contains a list of vertices
    def __init__(self, n_vertices):
        self.vertices = [None] * (n_vertices+1)
        self.num_vertices = len(self.vertices)
    
    # Returns the i'th node
    def getVertex(self, i):
        if ((i > len(self.vertices)) | (i <= 0)):
            raise ValueError(f'index {i} is out of bounds')
        else:
            return self.vertices[i]
    
    # Adds a new vertex to the graph
    def _addVertex(self, vertex_index, neighor_index, distance):
        if (self.vertices[vertex_index] == None):
            # Construct new vertex
            self.vertices[vertex_index] = Vertex(vertex_index, neighor_index, distance)
        else:
            # Vertex already in graph but other neighbor, add extra edge
            self.vertices[vertex_index]._addNeighbor(neighor_index, distance)


In [3]:
import fileinput

# Read graph data
def readGraph(filePath):
    n_vertices = 0
    for line in fileinput.input([filePath]):
        words = line.split(" ")
        if (words[0] == "p"):
            n_vertices = int(words[2])
    graph = Graph(n_vertices)
    for line in fileinput.input([filePath]):
        words = line.split(" ")
        if (words[0] == "a"):
            graph._addVertex(int(words[1]), int(words[2]), float(words[3]))
    return graph


# Read coordinates data
def readCoordinates(filepath):
    # Start to count from 1
    coordinates = [None]
    for line in fileinput.input([filepath]):
        words = line.split(" ")
        if (words[0] == "v"):
            coordinates.append([float(words[2]), float(words[3])])
    return coordinates


### Usefull functions

In [16]:
import numpy as np
    
# Priority queue definition
class PriorityQueue(dict):
    def put(self, item, value):
        # Watch out that value is not overwritten with higher value, shouldn't be allowed to happen!
        if item in self :
            value = min( value, self[item])
        self[item] = value
    def pop(self):
        """
        Returns the item with the lowest weight
        """
        item_min = min(self, key=self.get)
        super().pop(item_min)
        return item_min

    
def angles2centimeters(lo, la):
    """
    Convert longitude and latitude to local orthogonal grid
    :param lo: longitude
    :param la: latitude
    :return: height and width coordinates in cm's
    """
    
    radius = 6300 * 1e4  # cm
    la_mean = 40794234.  # 1e-6 degree
    lo_mean = -74016939.  # 1e-6 degree
    
    w = radius * np.cos(np.radians(la / 1e6)) * np.radians((lo - lo_mean) / 1e6)
    h = radius * np.radians((la - la_mean) / 1e6)
    
    return w, h 

# Assignment

## Code skeletons

Feel free to move the following code to the relevant questions. 

Before submitting your code, make sure to execute all code fields sequentially. Notebooks that don't execute sequentially will be penalised.

In [10]:
import math 

# The graph and coordinates data
# TODO: implement
graph = readGraph('USA-road-d.NY.gr.txt')
co = readCoordinates('USA-road-d.NY.co.txt')


# Heuristic function
def h(node1, node2):
    """
    Heuristic function
    """
    
    i1 = node1
    i2 = node2

    co1 = co[i1]
    co2 = co[i2]
    if not len(co1) == len(co2) :
        print( "Coordinates are not of the same dimensions." )

    EuclidDistSq = 0
    for i in range(0, len(co1)) : 
        EuclidDistSq += pow( co1[i] - co2[i], 2 )
    
    return math.sqrt( EuclidDistSq )

def printPath( cameFrom, start, goal ) :
    node = goal
    path = [goal]
    cost = 0
    while not (node == start) :
        previous = cameFrom[node]
        weights_prev = graph.getVertex(previous).getWeights()
        index =  graph.getVertex(previous).getNeighbors().index(node)
        cost += weights_prev[index]
        node = previous
        path.append(node)
    path.reverse()
    print(path)
    return (path, cost)


# Algorithm
def a_star_search(graph, co, start, goal):
    """
    A* algorithm
    :param graph: Graph object
    :param co: coordinates list
    :param start: index of start node
    :param goal: index of goal node
    :return: The path of nodes and total length
    """
    open = set([start]) # A set of nodes
    closed = set([]) # A set of nodes

    cameFrom = {} # A node-node key-value mapping

    gScore = {} # A node-int key-value mapping
    gScore[start] = 0

    fScore = {} # A node-int key-value mapping
    fScore[start] = gScore[start] + h(start, goal)

    nodeOrder = PriorityQueue()
    nodeOrder.put(start, fScore.get(start))

    while not len(open) == 0 :
        # node with lowest f value --> track with priority queue of nodes + key-value mapping (node-fScore)
        currentNode = nodeOrder.pop()
        #print( 'current node : ' +  str(currentNode))
        if currentNode == goal :
            #print( cameFrom )
            return printPath(cameFrom, start, goal)

        open.remove(currentNode)
        closed.add(currentNode)
        neighbours = graph.getVertex(currentNode).getNeighbors()
        #print( 'neighbours : ' +  str(neighbours))

        weights = graph.getVertex(currentNode).getWeights()
        
        for neighbour in neighbours :
            if not neighbour in closed :
                tentative_gScore = gScore[currentNode] + weights[ neighbours.index(neighbour) ]
                #print( 'gScore ' + str(gScore[currentNode] ))
                #print(' weight ' + str(weights[ neighbours.index(neighbour) ])) 
                
                if ( neighbour not in gScore.keys() ) or ( tentative_gScore < gScore[neighbour] ) :
                    #print('chenged')
                    #print( 'node ' +  str(neighbour) + ' added to cameFrom')

                    gScore[neighbour] = tentative_gScore
                    fScore[neighbour] = gScore[neighbour] + h(neighbour, goal)
                    cameFrom[neighbour] = currentNode 
                    if neighbour not in open :
                        open.add(neighbour)
                        nodeOrder.put(neighbour, fScore[neighbour])

                """
                if neighbour not in open :
                    open.add(neighbour)
                    fScore[neighbour] = float('inf')
                    gScore[neighbour] = float('inf')
                else :
                    print(' Hey, ' + str(gScore[neighbour]))
                    if tentative_gScore < gScore[neighbour] :
                        print('chenged')
                        #print( 'node ' +  str(neighbour) + ' added to cameFrom')
                        gScore[neighbour] = tentative_gScore
                        fScore[neighbour] = gScore[neighbour] + h(neighbour, goal)
                        cameFrom[neighbour] = currentNode 
                """
    return [], 0

In [18]:
# Calculate the path between your start and goal node. 
# Did you get the shortest-distance path? You can 
# verify your results in the distances.txt file.

import random

group_number = 10  # TODO: change to your group number
num_vertices = graph.num_vertices  # TODO: number of vertices in the graph

random.seed(group_number)

start = random.randint(1, num_vertices + 1)
goal = random.randint(1, num_vertices + 1)


# Calculating the path between nodes
print( 'start : ' +  str(start) )
print( 'goal : ' + str(goal) )
path, path_cost = a_star_search(graph, co, start, goal)
print('cost of path: ' + str(path_cost))
path_length = len(path)
print('length of path: ' + str(path_length))

start : 17084
goal : 224862
[17084, 17072, 17073, 17086, 17087, 17117, 17119, 17121, 17126, 17170, 17171, 17284, 17276, 17283, 17290, 17292, 17293, 17300, 17301, 17299, 17302, 17334, 17335, 17338, 17345, 17347, 17348, 17419, 17420, 17425, 17859, 17860, 17867, 17866, 17868, 17871, 17895, 34253, 18119, 17897, 17896, 18519, 18509, 18256, 18523, 18528, 18552, 18530, 18554, 18553, 18562, 18563, 18565, 18548, 18569, 18571, 18657, 18658, 18660, 18663, 18664, 18665, 18667, 18668, 18212, 18211, 18671, 18687, 18688, 18689, 18691, 18713, 18718, 18719, 18975, 18974, 18976, 18965, 18964, 18967, 18970, 18971, 25303, 25304, 25308, 25309, 25310, 25381, 25382, 25384, 25385, 25365, 25362, 25367, 25368, 25370, 25371, 25374, 25379, 25377, 25376, 21892, 21891, 26542, 26539, 26541, 26548, 26545, 26546, 26154, 26148, 26151, 26176, 26177, 26174, 26175, 26181, 26180, 26185, 19181, 11217, 26263, 26262, 26265, 26251, 26257, 26256, 26255, 26258, 26254, 26253, 26236, 26282, 26235, 26234, 26237, 26233, 26230, 26238

## Answers

Answer the questions from the assignment and add appropriate code where relevant to the question.

In [ ]:
# TODO

In [ ]:
# Question 1
graph = readGraph("USA-road-d.NY.gr.txt")
vertices_nr = graph.num_vertices
print(vertices_nr)

edges_nr = 0
for i in range(1, vertices_nr):
    edges_nr += len(graph.getVertex(i).getNeighbors())

print("The number of vertices is {}".format(vertices_nr))
print("The number of edges is {}".format(edges_nr))

264347
The number of vertices is 264347
The number of edges is 733846


In [ ]:
# Question 2
print("A* algorithm finds the cheapest path from a start node to the goal by keeping track of the current cheapest path from start node to node n in g(n) and searching for the best path from current node n to goal using a heuristic function h(n), e.g Euclidean distance. Adding up g(n) and h(n) results in the estimated cost of the cheapest path through node n, marked by f(n). The heuristic function should be consistent and admissible, consistent meaning that its estimate to the goal is always less than the estimate from any neighbouring node to the goal, plus the cost of reaching that neighbor, and admissible meaning that the function never overestimates the actual cost of reaching the goal.")

A* algorithm finds the cheapest path from a start node to the goal by keeping track of the current cheapest path from start node to node n in g(n) and searching for the best path from current node n to goal using a heuristic function h(n), e.g Euclidean distance. Adding up g(n) and h(n) results in the estimated cost of the cheapest path through node n, marked by f(n). The heuristic function should be consistent and admissible, consistent meaning that its estimate to the goal is always less than the estimate from any neighbouring node to the goal, plus the cost of reaching that neighbor, and admissible meaning that the function never overestimates the actual cost of reaching the goal.


In [ ]:
# Question 3
#-- See above --#

In [ ]:
# Question 4

In [ ]:
# Question 5

In [ ]:
# Queston 6

In [ ]:
# Question 7
graph = readGraph('USA-road-t.NY.gr.txt')
co = readCoordinates('USA-road-d.NY.co.txt')

In [ ]:
# Question 8
## Best heuristic somehow takes into account the weights (i.e. times). For example: Give weight to every node == sum of all it's (outgoing) arcs. use this weights a an additional weight in combination with euclidian heuristic.
def h_8(node1, node2):
    """
    Heuristic function
    """
    
    i1 = node1
    i2 = node2

    co1 = co[i1]
    co2 = co[i2]

    weight1 = graph.getVertex(i1).getWeigths()
    weight2 = graph.getVertex(i2).getWeigths()
    weight = (weight1 + weight2) /2
    if not len(co1) == len(co2) :
        print( "Coordinates are not of the same dimensions." )

    EuclidDistSq = 0
    for i in range(0, len(co1)) : 
        WeightedEuclidDistSq += weight*(pow( co1[i] - co2[i], 2 ))
    
    return math.sqrt( WeightedEuclidDistSq )
